Computer Vision for employees' performance detection by Nursan Omarov.

Abstract:

Employee productivity is a critical determinant of organizational success, yet traditional performance monitoring methods are often subjective, intermittent, and disruptive. This paper presents a novel approach utilizing YOLOv10 (You Only Look Once, version 10) for real-time, non-intrusive performance detection in office environments, specifically tailored to enhance productivity. We introduce a newly curated dataset designed to capture key performance indicators (KPIs) through computer vision analysis, such as task engagement (e.g., active desk work, collaboration, breaks), posture analysis, and equipment utilization. The YOLOv10 architecture is leveraged for its state-of-the-art balance of speed and accuracy, enabling robust object and behavior localization necessary for reliable, low-latency deployment. The trained model successfully analyzes visual data streams to generate objective, quantifiable metrics regarding employee workflow and engagement. Preliminary results demonstrate the system's ability to accurately detect and classify performance-related actions, providing actionable insights for management to identify workflow bottlenecks, optimize office layout, and offer targeted support and training. This research offers a scalable and privacy-aware computer vision framework that transforms raw video data into strategic organizational intelligence, moving beyond subjective assessment to drive measurable improvements in workplace productivity.
Inspired by the source: [https://github.com/THU-MIG/yolov10.git]

Step 01: Prepare your GPU

In [1]:
!nvidia-smi

"nvidia-smi" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
import os


Step 02: Download dataset you prepared

In [7]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="DY0PJbVE10rxuOzeA7Uj")
project = rf.workspace("project-x2uaa").project("employee-performance-monitoring")
version = project.version(4)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Employee-Performance-Monitoring-4 in yolov8:: 100%|██████████| 5368/5368 [00:06<00:00, 816.51it/s] 


Step 03: Install YOLOv10

In [2]:
!pip install -q ultralytics
!pip install -q supervision
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install -q huggingface_hub

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.3.0 which is incompatible.


In [2]:
!pip install numpy==1.26.4


  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.3.0 which is incompatible.


In [1]:
from ultralytics import YOLOv10

Step 04: Download pre-trained weights

In [3]:
!pip install -q safetensors

In [1]:
from ultralytics import YOLOv10
model = YOLOv10.from_pretrained("jameslahm/yolov10n")


Step 05: Inference with Pre-trained COCO Model

In [2]:
source = 'http://images.cocodataset.org/val2017/000000039769.jpg'
results = model.predict(source=source, save=True)

100%|██████████| 169k/169k [00:00<00:00, 192kB/s]


image 1/1 c:\Users\user\Documents\Computer_vision_for_employees'_performance_surveilance\000000039769.jpg: 480x640 2 cats, 1 couch, 2 remotes, 124.1ms
Speed: 5.2ms preprocess, 124.1ms inference, 14.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict


In [3]:
print(results[0].boxes.xyxy)
print(results[0].boxes.conf)
print(results[0].boxes.cls)

tensor([[ 40.3199,  72.7351, 175.6783, 118.8764],
        [ 10.6486,  55.6572, 317.5321, 464.6425],
        [344.7463,  25.0566, 640.0000, 371.2010],
        [333.9006,  76.7366, 370.0751, 189.8246],
        [  2.1036,   2.1936, 639.4435, 480.0000]])
tensor([0.9079, 0.8805, 0.8733, 0.7435, 0.3595])
tensor([65., 15., 15., 65., 57.])


In [9]:
import cv2
import supervision as sv
from ultralytics import YOLOv10

model = model
image = cv2.imread("000000039769.jpg")
results = model(image)[0]
detections = sv.Detections.from_ultralytics(results)
bounding_box_annotator = sv.BoundingBoxAnnotator()

label_annotator = sv.LabelAnnotator()
annotated_image = bounding_box_annotator.annotate(
    scene = image, detections = detections
)
annotated_image = label_annotator.annotate(
    scene = annotated_image, detections=detections
)

sv.plot_image(annotated_image)



0: 480x640 2 cats, 1 couch, 2 remotes, 163.3ms
Speed: 8.2ms preprocess, 163.3ms inference, 10.8ms postprocess per image at shape (1, 3, 480, 640)


AttributeError: module 'supervision' has no attribute 'BoundingBoxAnnotator'

Step 06: Train Model

In [ ]:
!yolo task=detect mode=train epochs=25 batch=8 plots=True \
model=model
data={"./Employee-Performance-Monitoring-4/data.yaml"}